# Import Data

In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('../Data/world_athletics_long_jump.csv')

In [8]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217398 entries, 0 to 217397
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Rank         217398 non-null  int64  
 1   Mark         217398 non-null  float64
 2   Wind         144075 non-null  float64
 3   Competitor   217398 non-null  object 
 4   DOB          183320 non-null  object 
 5   Nat          217398 non-null  object 
 6   Pos          206628 non-null  object 
 7   Venue        217398 non-null  object 
 8   Date         217398 non-null  object 
 9   Score        217398 non-null  int64  
 10  Gender       217398 non-null  object 
 11  Year         217398 non-null  int64  
 12  Profile_URL  217398 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 21.6+ MB


,Rank,Mark,Wind,Competitor,DOB,Nat,Pos,Venue,Date,Score,Gender,Year,Profile_URL
0,1,8.43,NaN,Iván PEDROSO,17-Dec-72,CUB,1,"Atlantic Pavillion, Lisboa (POR) (i)",11-Mar-01,1232,men,2001,https://worldathletics.org/athletes/athlete=14...
1,2,8.41,1.3,James BECKFORD,9-Jan-75,JAM,1,Torino (ITA),9-Jun-01,1227,men,2001,https://worldathletics.org/athletes/athlete=14...
2,3,8.40,1.2,Iván PEDROSO,17-Dec-72,CUB,1,"Commonwealth Stadium, Edmonton (CAN)",11-Aug-01,1225,men,2001,https://worldathletics.org/athletes/athlete=14...
3,4,8.38,0.6,Savanté STRINGFELLOW,6-Nov-78,USA,1,"Orlando, FL (USA)",22-Jul-01,1221,men,2001,https://worldathletics.org/athletes/athlete=14...
4,5,8.33,-0.1,Savanté STRINGFELLOW,6-Nov-78,USA,1q2,"Commonwealth Stadium, Edmonton (CAN)",9-Aug-01,1210,men,2001,https://worldathletics.org/athletes/athlete=14...


# Cleaning The Data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217398 entries, 0 to 217397
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Rank         217398 non-null  int64  
 1   Mark         217398 non-null  float64
 2   Wind         144075 non-null  float64
 3   Competitor   217398 non-null  object 
 4   DOB          183320 non-null  object 
 5   Nat          217398 non-null  object 
 6   Pos          206628 non-null  object 
 7   Venue        217398 non-null  object 
 8   Date         217398 non-null  object 
 9   Score        217398 non-null  int64  
 10  Gender       217398 non-null  object 
 11  Year         217398 non-null  int64  
 12  Profile_URL  217398 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 21.6+ MB


Remove duplicates

In [10]:
df = df.drop_duplicates()

Cleaning `Date of Bearth (DOB)` and `Date` columns

In [11]:
df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

C:\Users\mhahi\AppData\Local\Temp\ipykernel_25400\2247441291.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')
C:\Users\mhahi\AppData\Local\Temp\ipykernel_25400\2247441291.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


Create `Age` column

In [12]:
age = df['Date'].dt.year - df['DOB'].dt.year

# subtract 1 if birthday not reached yet in event year
before_birthday = (
    (df['Date'].dt.month < df['DOB'].dt.month) |
    ((df['Date'].dt.month == df['DOB'].dt.month) &
     (df['Date'].dt.day < df['DOB'].dt.day))
)

df['Age'] = age - before_birthday.astype('Int64')

# Remove age wrong data and outliers
df = df.drop(df[(df['Age'] < 16) | (df['Age'] > 40)].index)

Set age category

In [13]:
def set_age_category(age):
    if age < 18:
        return 'u18'
    elif age < 20:
        return '20'
    elif age >= 20:
        return 'seniors'
    else: return np.nan

In [14]:
df['Age Category'] = df['Age'].apply(set_age_category)

Reoreder the columns

In [15]:
new_order = [
    'Rank', 'Mark', 'Score', 'Pos',
    'Competitor', 'Nat', 'DOB', 'Age', 'Age Category',
    'Venue', 'Date', 'Year'
    ]
df = df[new_order]

Final DataFrame

In [16]:
df.info()
df.sort_values('Mark', ascending=False)

<class 'pandas.core.frame.DataFrame'>
Index: 211813 entries, 3 to 217397
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Rank          211813 non-null  int64         
 1   Mark          211813 non-null  float64       
 2   Score         211813 non-null  int64         
 3   Pos           201202 non-null  object        
 4   Competitor    211813 non-null  object        
 5   Nat           211813 non-null  object        
 6   DOB           177735 non-null  datetime64[ns]
 7   Age           177735 non-null  Float64       
 8   Age Category  177735 non-null  object        
 9   Venue         211813 non-null  object        
 10  Date          211813 non-null  datetime64[ns]
 11  Year          211813 non-null  int64         
dtypes: Float64(1), datetime64[ns](2), float64(1), int64(3), object(5)
memory usage: 21.2+ MB


,Rank,Mark,Score,Pos,Competitor,Nat,DOB,Age,Age Category,Venue,Date,Year
14719,1,8.74,1307,1,Dwight PHILLIPS,USA,1977-10-01,31.0,seniors,"Eugene, OR (USA)",2009-06-07,2009
13018,1,8.73,1298,1,Irving SALADINO,PAN,1983-01-23,25.0,seniors,"FBK Stadium, Hengelo (NED)",2008-05-24,2008
14720,2,8.71,1293,1,Sebastian BAYER,GER,1986-06-11,22.0,seniors,"Oval Lingotto, Torino (ITA) (i)",2009-03-08,2009
95477,1,8.69,1289,1,Tajay GAYLE,JAM,1996-08-02,23.0,seniors,"Khalifa International Stadium, Doha (QAT)",2019-09-28,2019
86607,1,8.68,1287,1,Juan Miguel ECHEVARRÍA,CUB,1998-08-11,19.0,20,Bad Langensalza (GER),2018-06-30,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
217391,4809,6.30,773,8,Kristjan SILD,EST,2004-03-19,21.0,seniors,"Lasnamäe Kergejõustikuhall, Tallinn (EST) (i)",2026-02-07,2026
217389,4809,6.30,773,12f2,Noah HOPES,USA,NaT,<NA>,NaN,"Ashenfelter Indoor Track, State College, PA (U...",2026-02-07,2026
217388,4809,6.30,773,10,Mateus ALVES,BRA,2010-01-01,16.0,u18,"Altice Forum, Braga (POR) (i)",2026-02-07,2026
217387,4809,6.30,773,2,Aksel WORMDAHL,NOR,2006-02-01,20.0,seniors,"Rud friidrettstadion, Bærum (NOR) (i)",2026-02-07,2026


In [17]:
df.to_csv('../Data/cleaned_long_Jump.csv', index=False)